In [1]:
from database_utils import DatabaseConnector
from data_extraction import DataExtractor
from data_cleaning import DataCleaning
import numpy as np
import pandas as pd

In [2]:
# Initiate DatabseConnector class to connect to SQL database containing the raw data.
db_raw = DatabaseConnector('credentials/db_creds_raw.yaml')

# Initiate DataExtractor class to extract data currently in raw SQL database.
raw_data = DataExtractor('credentials/db_creds_raw.yaml')

In [3]:
# List tables from raw SQL database.
tables_list = raw_data.db_connector.list_db_tables()
print(f'Tables in raw data: {tables_list}')

Tables in raw data: ['dim_card_details', 'legacy_store_details', 'legacy_users', 'orders_table']


In [4]:
# Extract all raw data tables.
user_data = raw_data.read_rds_table('legacy_users')
card_data = raw_data.read_rds_table('dim_card_details')
store_data = raw_data.read_rds_table('legacy_store_details')
product_data = raw_data.extract_from_s3(url_path = 's3://data-handling-public/products.csv', new_file_name = 'product_data')
orders_table = raw_data.read_rds_table('orders_table')
date_details = raw_data.extract_from_s3(url_path = 's3://data-handling-public/date_details.json', new_file_name = 'date_details')

In [5]:
# Initiate DataCleaning class and clean each data set accordingly.
cleaner = DataCleaning()
user_data_cleaned = cleaner.clean_user_data(user_data)
card_data_cleaned = cleaner.clean_card_data(card_data)
store_data_cleaned = cleaner.clean_store_data(store_data)
product_data_cleaned = cleaner.clean_products_data(product_data)
orders_table_cleaned = cleaner.clean_orders_data(orders_table)
date_details_cleaned = cleaner.clean_dates_details(date_details)

In [6]:
db_cleaned = DatabaseConnector('credentials/db_creds_cleaned.yaml')
db_cleaned.clear_database()

All tables have been removed from SQL database.


In [7]:
# Initiate DatabaseConnector class to connect to SQL database that will contain new cleaned data and
# upload cleaned datasets to this database.
db_cleaned = DatabaseConnector('credentials/db_creds_cleaned.yaml')
db_cleaned.upload_to_db(user_data_cleaned, table_name='dim_users', if_exists='replace')
db_cleaned.upload_to_db(card_data_cleaned, table_name='dim_card_details', if_exists='replace')
db_cleaned.upload_to_db(store_data_cleaned, table_name='dim_store_details', if_exists='replace')
db_cleaned.upload_to_db(product_data_cleaned, table_name='dim_products', if_exists='replace')
db_cleaned.upload_to_db(orders_table_cleaned, table_name='orders_table', if_exists='replace')
db_cleaned.upload_to_db(date_details_cleaned, table_name='dim_date_times', if_exists='replace')

In [9]:
# Clean tables, update data types and add primary and foreign keys in cleaned SQL database,
# and list tables within this database.
db_cleaned.execute_sql_from_folder('SQL_tidy_commands')
db_cleaned.list_db_tables()

All SQL commands executed.


['dim_users',
 'dim_card_details',
 'dim_store_details',
 'dim_products',
 'orders_table',
 'dim_date_times']